# TP – API “Digital Social Score”

De l’analyse de texte à l’infrastructure Cloud sécurisée, scalable 
et conform

## Etape 1 : Exploration, analyse et anonymisation des données

###  Téléchargons le dataset **Toxic Comment Classification Datase**(Hugging Face)

In [1]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("thesofakillers/jigsaw-toxic-comment-classification-challenge", split="train")

# Afficher les premier lignes
df = pd.DataFrame(dataset)
df


c:\Users\xadee\OneDrive\Bureau\API\API-Digital-Social-Score\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


### Analyse de la structure du dataset

Analysons la structure, les dimensions et la distribution des labels du dataset Toxic Comment Classification.

In [ ]:
# 1. DIMENSIONS
print(f"\n📏 DIMENSIONS:")
print(f"   Nombre de lignes: {len(df):,}")
print(f"   Nombre de colonnes: {len(df.columns)}")
print(f"   Colonnes: {list(df.columns)}")

# 2. TYPES DE DONNÉES
print(f"\n📋 TYPES DE DONNÉES:")
print(df.dtypes)

# 3. VALEURS MANQUANTES
print(f"\n🔍 VALEURS MANQUANTES:")
missing = df.isnull().sum()
if missing.sum() == 0:
    print("   ✅ Aucune valeur manquante")
else:
    print(missing[missing > 0])

# 4. DISTRIBUTION DES LABELS
print(f"\n📊 DISTRIBUTION DES LABELS (% de commentaires toxiques):")
label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
for col in label_columns:
    count = df[col].sum()
    percentage = (count / len(df)) * 100
    print(f"   {col:15s}: {count:6,} ({percentage:5.2f}%)")

# 5. STATISTIQUES SUR LA LONGUEUR DES TEXTES
df['text_length'] = df['comment_text'].astype(str).apply(len)
df['word_count'] = df['comment_text'].astype(str).apply(lambda x: len(x.split()))

print(f"\n📝 STATISTIQUES SUR LES TEXTES:")
print(f"   Longueur moyenne: {df['text_length'].mean():.0f} caractères")
print(f"   Longueur médiane: {df['text_length'].median():.0f} caractères")
print(f"   Nombre de mots moyen: {df['word_count'].mean():.0f} mots")

# 6. EXEMPLES DE COMMENTAIRES
print(f"\n💬 EXEMPLES DE COMMENTAIRES:")
print(f"\n   Non-toxique:")
non_toxic = df[df['toxic'] == 0]['comment_text'].iloc[0]
print(f"   {non_toxic[:150]}...")

print(f"\n   Toxique:")
toxic = df[df['toxic'] == 1]['comment_text'].iloc[0]
print(f"   {toxic[:150]}...")

# 7. JUSTIFICATION DU CHOIX DE 10K ÉCHANTILLONS
print(f"\n🎯 JUSTIFICATION DU CHOIX DE 10,000 ÉCHANTILLONS:")
print(f"   ✓ Dataset complet: {len(df):,} commentaires (très volumineux)")
print(f"   ✓ Échantillon de 10k: représentatif et gérable")
print(f"   ✓ Permet un entraînement rapide pour le TP")
print(f"   ✓ Conservation de la distribution des classes")
print(f"   ✓ Conformité RGPD: limitation des données traitées")

### Utilisons spaCy pour anonymiser les données 

In [2]:
# python -m spacy download en_core_web_sm"
import spacy
import re
import pandas as pd
from tqdm import tqdm

# Charger spaCy pour l'anglais (une seule fois)
nlp = spacy.load("en_core_web_sm")

# Patterns regex pour détecter les données personnelles
EMAIL_PATTERN = re.compile(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b')
PHONE_PATTERN = re.compile(r'\b(?:\+?1[-.\s]?)?\(?[0-9]{3}\)?[-.\s]?[0-9]{3}[-.\s]?[0-9]{4}\b')

def detect_personal_data(text):
    """
    Détecte toutes les données personnelles pour la conformité RGPD
    """
    if not text or pd.isna(text):
        return {
            'has_personal_data': False,
            'names': [],
            'emails': [],
            'phones': [],
            'addresses': []
        }
    
    text_str = str(text)
    
    # 1. EMAILS
    emails = EMAIL_PATTERN.findall(text_str)
    
    # 2. TÉLÉPHONES  
    phones = PHONE_PATTERN.findall(text_str)
    
    # 3. NOMS avec spaCy NER
    doc = nlp(text_str)
    names = []
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            names.append(ent.text)
    
    return {
        'has_personal_data': bool(names or emails or phones),
        'names': names,
        'emails': emails,
        'phones': phones,
    }

def anonymize_text(text, detection_result):
    """
    Anonymise automatiquement le texte pour la conformité RGPD
    """
    if not detection_result['has_personal_data']:
        return text
    
    anonymized_text = str(text)
    
    for email in detection_result['emails']:
        anonymized_text = anonymized_text.replace(email, "[EMAIL]")
    
    for phone in detection_result['phones']:
        anonymized_text = anonymized_text.replace(phone, "[PHONE]")
    
    for name in detection_result['names']:
        anonymized_text = anonymized_text.replace(name, "[NAME]")
    
    return anonymized_text


In [ ]:
# CRÉATION DU DATASET ANONYMISÉ (df_anonymized)
print("ANONYMISATION DU DATASET")
print("=" * 30)

# Travaillons avec 10 000 données
max_to_anonymize = 10000

# Taille réelle à traiter
actual_size = min(max_to_anonymize, len(df))

print(f"PARAMETRES D'ANONYMISATION:")
print(f"Dataset total: {len(df):,} commentaires")
print(f"A anonymiser: {actual_size:,} commentaires")

# Créer une copie du DataFrame pour l'anonymisation
df_anonymized = df.copy()

# Statistiques de traitement
comments_with_personal_data = 0
total_anonymizations = 0

print(f"\nAnonymisation en cours...")

# Traiter les commentaires avec barre de progression
for i in tqdm(range(actual_size), desc="Anonymisation"):
    original_comment = df['comment_text'].iloc[i]
    
    # Détecter les données personnelles
    detection = detect_personal_data(original_comment)
    
    # Si des données personnelles sont trouvées, anonymiser
    if detection['has_personal_data']:
        comments_with_personal_data += 1
        
        # Compter le nombre total d'éléments à anonymiser
        num_elements = (len(detection['names']) + len(detection['emails']) + len(detection['phones']))
        total_anonymizations += num_elements
        
        # Anonymiser le texte
        anonymized_comment = anonymize_text(original_comment, detection)
        
        # Remplacer dans le DataFrame anonymisé
        df_anonymized.loc[i, 'comment_text'] = anonymized_comment

print(f"Total commentaires traités: {actual_size:,}")
print(f"Commentaires anonymisés: {comments_with_personal_data:,}")
print(f"Pourcentage de commentaires affectés: {comments_with_personal_data/actual_size*100:.2f}%")
print(f"Total d'éléments anonymisés: {total_anonymizations:,}")

# Sauvegarder le dataset anonymisé
anonymized_filename = "dataset_anonymized_rgpd_10k.csv"
df_anonymized.to_csv(anonymized_filename, index=False)
print(f"\nDataset anonymisé sauvegardé: {anonymized_filename}")

### Comparaison AVANT/APRÈS Anonymisation

Visualisons des exemples concrets d'anonymisation.

In [ ]:
# Trouver des commentaires avec données personnelles
examples_to_show = []
for i in range(min(10000, len(df))):
    original = df['comment_text'].iloc[i]
    detection = detect_personal_data(original)
    
    if detection['has_personal_data']:
        anonymized = anonymize_text(original, detection)
        examples_to_show.append({
            'index': i,
            'original': original,
            'anonymized': anonymized,
            'detection': detection
        })
        
        if len(examples_to_show) >= 10:  # Afficher 10 exemples
            break

# Afficher les exemples
for idx, example in enumerate(examples_to_show, 1):
    print(f"\n EXEMPLE {idx}:")
    print(f"   Index: {example['index']}")
    print(f"\n    AVANT (données personnelles détectées):")
    print(f"      {example['original'][:200]}...")
    print(f"\n   Détections:")
    if example['detection']['names']:
        print(f"      - Noms: {example['detection']['names']}")
    if example['detection']['emails']:
        print(f"      - Emails: {example['detection']['emails']}")
    if example['detection']['phones']:
        print(f"      - Téléphones: {example['detection']['phones']}")
    print(f"\n    APRÈS (anonymisé):")
    print(f"      {example['anonymized'][:200]}...")
    print("-" * 70)

print(f"\n Total d'exemples trouvés: {len(examples_to_show)}")

In [ ]:
# CRÉATION DU TABLEAU RÉCAPITULATIF
import pandas as pd

comparison_data = []
for example in examples_to_show[:5]:  # Top 5 pour le tableau
    comparison_data.append({
        'Index': example['index'],
        'Noms': len(example['detection']['names']),
        'Emails': len(example['detection']['emails']),
        'Téléphones': len(example['detection']['phones']),
        'Total Anonymisations': (
            len(example['detection']['names']) + 
            len(example['detection']['emails']) + 
            len(example['detection']['phones'])
        )
    })

df_comparison = pd.DataFrame(comparison_data)
print("\nTABLEAU RÉCAPITULATIF DES ANONYMISATIONS:")
print(df_comparison.to_string(index=False))

###  Justification des choix d'anonymisation

**Pourquoi ces méthodes ?**

1. **NER (Named Entity Recognition) avec spaCy** :
   - Détection automatique des noms de personnes
   - Précision élevée pour l'anglais
   - Modèle pré-entraîné robuste

2. **Regex pour emails et téléphones** :
   - Patterns standards reconnus
   - Complémentaire au NER
   - Haute sensibilité

3. **Remplacement par tokens** :
   - `[NAME]`, `[EMAIL]`, `[PHONE]` : tokens génériques
   - Préserve la structure du texte
   - Permet l'analyse sans données personnelles
   - Anonymisation irréversible

**Conformité RGPD :**
- Article 4(5) : Pseudonymisation effective
- Article 25 : Privacy by design
- Article 30 : Registre des traitements
- Article 32 : Sécurité du traitement

### Comparons la version initiale et anonymisée et justifions chaque choix.

## Etape 2 :  Préparation et entraînement d’un modèle IA

### Nettoyage des textes (ponctuation, emojis, casse)

In [ ]:
import re
import string
import pandas as pd
from tqdm import tqdm

def clean_text(text):
    """
    Fonction complète de nettoyage de texte pour le NLP
    """
    if not text or pd.isna(text):
        return ""
    
    # Convertir en string au cas où
    text = str(text)
    
    # 1. CASSE : Convertir en minuscules
    text = text.lower()
    
    # 2. EMOJIS : Supprimer les emojis (patterns Unicode)
    emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symboles & pictogrammes
                              u"\U0001F680-\U0001F6FF"  # transport & cartes
                              u"\U0001F1E0-\U0001F1FF"  # drapeaux
                              u"\U00002500-\U00002BEF"  # caractères chinois
                              u"\U00002702-\U000027B0"
                              u"\U00002702-\U000027B0"
                              u"\U000024C2-\U0001F251"
                              u"\U0001f926-\U0001f937"
                              u"\U00010000-\U0010ffff"
                              u"\u2640-\u2642"
                              u"\u2600-\u2B55"
                              u"\u200d"
                              u"\u23cf"
                              u"\u23e9"
                              u"\u231a"
                              u"\ufe0f"  # dingbats
                              u"\u3030"
                              "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    # 3. CARACTÈRES SPÉCIAUX : Supprimer les caractères non-ASCII
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    
    # 4. URLs : Supprimer les URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', text)
    text = re.sub(r'www\.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', text)
    
    # 5. MENTIONS : Supprimer les @mentions
    text = re.sub(r'@\w+', ' ', text)
    
    # 6. HASHTAGS : Supprimer les #hashtags
    text = re.sub(r'#\w+', ' ', text)
    
    # 7. CHIFFRES : Supprimer les chiffres isolés
    text = re.sub(r'\b\d+\b', ' ', text)
    
    # 8. PONCTUATION : Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # 9. ESPACES : Normaliser les espaces multiples
    text = re.sub(r'\s+', ' ', text)
    
    # 10. TRIM : Supprimer les espaces en début/fin
    text = text.strip()
    
    return text

In [ ]:
# ÉTAPE 3: Application du nettoyage au dataset anonymisé
print("NETTOYAGE DU DATASET ANONYMISE")
print("=" * 35)

# Vérifier que df_anonymized existe
if 'df_anonymized' not in locals():
    print("df_anonymized n'existe pas encore!")
else:
    # Travailler seulement sur les 10 000 données anonymisées
    max_to_clean = 10000
    actual_size_to_clean = min(max_to_clean, len(df_anonymized))
    
    # Créer une copie du dataset anonymisé pour le nettoyage (seulement les 10k)
    df_cleaned = df_anonymized.head(actual_size_to_clean).copy()
    
    # Statistiques de nettoyage
    total_comments = len(df_cleaned)
    original_total_words = 0
    cleaned_total_words = 0
    empty_after_cleaning = 0
    
    print(f"Nettoyage de {total_comments:,} commentaires anonymisés...")
    
    # Appliquer le nettoyage avec barre de progression
    for i in tqdm(range(total_comments), desc="Nettoyage des commentaires"):
        original_comment = df_cleaned['comment_text'].iloc[i]
        
        if original_comment and not pd.isna(original_comment):
            # Compter les mots avant nettoyage
            original_total_words += len(str(original_comment).split())
            
            # Appliquer le nettoyage standard
            cleaned_comment = clean_text(original_comment)
            
            # Remplacer dans le DataFrame
            df_cleaned.loc[i, 'comment_text'] = cleaned_comment
            
            # Compter les mots après nettoyage
            if cleaned_comment:
                cleaned_total_words += len(cleaned_comment.split())
            else:
                empty_after_cleaning += 1
    

    print(f"Total commentaires traités: {total_comments:,}")
    if original_total_words > 0:
        print(f"Réduction: {((original_total_words - cleaned_total_words) / original_total_words * 100):.1f}%")
    print(f"Commentaires vides après nettoyage: {empty_after_cleaning}")
    
    # Supprimer les commentaires vides
    df_cleaned = df_cleaned[df_cleaned['comment_text'].str.len() > 0]
    print(f"Commentaires conservés: {len(df_cleaned):,}")
    
    # Sauvegarder le dataset nettoyé
    cleaned_filename = "dataset_cleaned_and_anonymized_10k.csv"
    df_cleaned.to_csv(cleaned_filename, index=False)
    print(f"\nDataset nettoyé sauvegardé: {cleaned_filename}")

### Entraînons le modèle avec SVM

In [ ]:
# Vérifier que df_cleaned existe
if 'df_cleaned' not in locals():
    print("ERREUR: df_cleaned n'existe pas!")
    print("Veuillez d'abord exécuter les étapes précédentes.")
else:
    # Sélectionner uniquement les colonnes nécessaires
    required_columns = ['comment_text', 'toxic']
    
    # Vérifier que les colonnes existent
    available_columns = list(df_cleaned.columns)
    missing_columns = [col for col in required_columns if col not in available_columns]
    
    if missing_columns:
        print(f"ATTENTION: Colonnes manquantes: {missing_columns}")
        # Utiliser les colonnes disponibles
        final_columns = [col for col in required_columns if col in available_columns]
    else:
        final_columns = required_columns
    
    print(f"Colonnes finales utilisées: {final_columns}")
    
    # Créer le dataset final
    df_final = df_cleaned[final_columns].copy()
    
   

### Split Train/Test

Divisons le dataset en ensembles d'entraînement et de test tout en préservant la distribution des classes.

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Vérifier que df_final existe
if 'df_final' not in locals():
    print(" ERREUR: df_final n'existe pas!")
    print("Veuillez exécuter les cellules précédentes.")
else:
    # Extraire X (textes) et y (labels)
    X = df_final['comment_text'].values
    y = df_final['toxic'].values
    
    # Split stratifié 80/20
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2,     
        random_state=42, 
        stratify=y          # Préserver la distribution des classes
    )
    
    print(f"\n RÉSULTATS DU SPLIT:")
    print(f"   Dataset total: {len(df_final):,} commentaires")
    print(f"   Train: {len(X_train):,} commentaires ({len(X_train)/len(df_final)*100:.1f}%)")
    print(f"   Test:  {len(X_test):,} commentaires ({len(X_test)/len(df_final)*100:.1f}%)")
    
    # Sauvegarder les datasets
    train_df = pd.DataFrame({'comment_text': X_train, 'toxic': y_train})
    test_df = pd.DataFrame({'comment_text': X_test, 'toxic': y_test})
    
    train_filename = "data/train_toxic_10k.csv"
    test_filename = "data/test_toxic_10k.csv"
    
    train_df.to_csv(train_filename, index=False)
    test_df.to_csv(test_filename, index=False)
    
    print(f"\n FICHIERS SAUVEGARDÉS:")
    print(f"   Train: {train_filename}")
    print(f"   Test:  {test_filename}")
    print("\n Split terminé avec succès!")

### Entraînement du Modèle SVM

Lançons l'entraînement du modèle SVM avec le pipeline TF-IDF.

In [ ]:
import subprocess
import os

# Vérifier que le fichier d'entraînement existe
train_file = "data/train_toxic_10k.csv"
if not os.path.exists(train_file):
    print(f" ERREUR: {train_file} n'existe pas!")
    print("Veuillez exécuter la cellule précédente (Split Train/Test).")
else:
    print(f" Fichier d'entraînement trouvé: {train_file}")
    print(f"\n Lancement de l'entraînement SVM...")
    print("-" * 60)
    
    # Exécuter le script SVM.py
    try:
        result = subprocess.run(
            ["python", "model/SVM.py"],
            capture_output=True,
            text=True,
            timeout=300  # Timeout de 5 minutes
        )
        
        # Afficher la sortie
        if result.stdout:
            print(result.stdout)
        
        if result.returncode == 0:
            print("\n ENTRAÎNEMENT SVM TERMINÉ AVEC SUCCÈS!")
            
            # Vérifier que le modèle a été sauvegardé
            model_path = "model/svm_pipeline.pkl"
            if os.path.exists(model_path):
                model_size = os.path.getsize(model_path) / (1024 * 1024)  # MB
                print(f" Modèle sauvegardé: {model_path} ({model_size:.2f} MB)")
            else:
                print(f"  Attention: {model_path} non trouvé")
        else:
            print("\n ERREUR lors de l'entraînement:")
            if result.stderr:
                print(result.stderr)
                
    except subprocess.TimeoutExpired:
        print(" TIMEOUT: L'entraînement a pris trop de temps (>5 min)")
    except Exception as e:
        print(f" ERREUR: {e}")

###  Évaluation du modèle SVM

Évaluons les performances du modèle SVM sur l'ensemble de test avec métriques détaillées et visualisations.

In [ ]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns

# ÉVALUATION DU MODÈLE SVM
print("=" * 60)
print(" ÉVALUATION DU MODÈLE SVM")
print("=" * 60)

# Charger le modèle
model_path = "model/svm_pipeline.pkl"
if not os.path.exists(model_path):
    print(f" ERREUR: Modèle non trouvé: {model_path}")
    print("Veuillez d'abord entraîner le modèle (cellule précédente).")
else:
    # Charger le modèle
    with open(model_path, 'rb') as f:
        svm_model = pickle.load(f)
    print(f" Modèle chargé: {model_path}")
    
    # Charger les données de test
    test_df = pd.read_csv("data/test_toxic_10k.csv")
    X_test = test_df['comment_text'].values
    y_test = test_df['toxic'].values
    
    print(f" Données de test chargées: {len(X_test)} commentaires")
    
    # Prédictions
    print(f"\n Prédictions en cours...")
    y_pred = svm_model.predict(X_test)
    
    # Calculer les métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
    
    print(f"\n MÉTRIQUES DE PERFORMANCE:")
    print(f"   Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"   Precision: {precision:.4f}")
    print(f"   Recall:    {recall:.4f}")
    print(f"   F1-Score:  {f1:.4f}")
    
    # Classification report détaillé
    print(f"\n CLASSIFICATION REPORT DÉTAILLÉ:")
    print(classification_report(y_test, y_pred, target_names=['Non-toxic', 'Toxic']))
    
    # Matrice de confusion
    cm = confusion_matrix(y_test, y_pred)
    
    print(f"\n MATRICE DE CONFUSION:")
    print(f"                  Prédit Non-toxic  Prédit Toxic")
    print(f"Réel Non-toxic         {cm[0][0]:6d}        {cm[0][1]:6d}")
    print(f"Réel Toxic             {cm[1][0]:6d}        {cm[1][1]:6d}")
    
    # Visualiser la matrice de confusion
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Non-toxic', 'Toxic'],
                yticklabels=['Non-toxic', 'Toxic'])
    plt.title('Matrice de Confusion - SVM')
    plt.ylabel('Vraie Classe')
    plt.xlabel('Classe Prédite')
    plt.tight_layout()
    plt.savefig('svm_confusion_matrix.png', dpi=100, bbox_inches='tight')
    plt.show()
    
    print(f"\n Matrice de confusion sauvegardée: svm_confusion_matrix.png")
    print(f"\n ÉVALUATION TERMINÉE!")

###  Tests de Prédiction - Exemples Concrets

Testons le modèle SVM sur des exemples concrets pour vérifier son comportement.

In [ ]:
# Charger le modèle si nécessaire
if 'svm_model' not in locals():
    with open("model/svm_pipeline.pkl", 'rb') as f:
        svm_model = pickle.load(f)

# Exemples de test
test_examples = [
    # Non-toxiques
    "This is a great article, thank you for sharing!",
    "I respectfully disagree with your point of view.",
    "Can you please explain this concept in more detail?",
    
    # Toxiques
    "You are stupid and your opinion is worthless!",
    "I hate you and everything you stand for.",
    "Shut up, nobody cares about what you think!",
    
    # Cas limites
    "This is ridiculous.",
    "I don't like this approach.",
    "You're wrong about this.",
]

print(f"\n Test sur {len(test_examples)} exemples:\n")

for i, text in enumerate(test_examples, 1):
    # Prédiction
    prediction = svm_model.predict([text])[0]
    label = " TOXIC" if prediction == 1 else " NON-TOXIC"
    
    # Obtenir le score de décision (confiance)
    try:
        decision_score = svm_model.decision_function([text])[0]
        confidence = abs(decision_score)
    except:
        confidence = 0
    
    print(f"{i:2d}. {label} (confiance: {confidence:.2f})")
    print(f"    Texte: \"{text}\"")
  

## Étape 3 : Déploiement du modèle en API Cloud

### Création d'une API Fast API recevant un texte et renvoyant un score.

###  Architecture de l'API FastAPI

Notre API de détection de toxicité est construite avec **FastAPI** et déployée sur **Google Kubernetes Engine (GKE)**.

**Endpoints principaux :**

1. **`POST /token`** : Authentification JWT
   - Entrée : `username` et `password`
   - Sortie : Token JWT valide 24h

2. **`POST /predict`** : Prédiction de toxicité
   - Entrée : `text` (commentaire à analyser)
   - Sortie : `score` (0-100) et `is_toxic` (booléen)
   - Authentification : JWT requis

3. **`GET /health`** : Vérification de santé
   - Sortie : État de l'API et du modèle

4. **`GET /metrics`** : Métriques Prometheus
   - Sortie : Métriques pour supervision

**Sécurité :**
- Authentification JWT (python-jose)
- Validation des entrées (pydantic)
- Rate limiting
- CORS configuré

### Test de l'API en Local

Testons l'API en local (localhost:8080) pour vérifier son fonctionnement.

In [ ]:
import requests
import json

# CONFIGURATION
API_URL = "http://localhost:8080"  # API locale
USERNAME = "admin"  
PASSWORD = "admin"

# ÉTAPE 1: Obtenir un token JWT
print("\n ÉTAPE 1: Authentification JWT")

try:
    token_response = requests.post(
        f"{API_URL}/token",
        data={
            "username": USERNAME,
            "password": PASSWORD
        },
        timeout=10
    )
    
    if token_response.status_code == 200:
        token_data = token_response.json()
        access_token = token_data['access_token']
        print(f" Token obtenu avec succès!")
        print(f"   Token type: {token_data['token_type']}")
        print(f"   Token: {access_token[:50]}...")
    else:
        print(f" Erreur d'authentification: {token_response.status_code}")
        print(f"   Réponse: {token_response.text}")
        access_token = None
        
except requests.exceptions.ConnectionError:
    print(f" ERREUR: Impossible de se connecter à {API_URL}")
    print(f"   Assurez-vous que l'API est démarrée (python app.py)")
    access_token = None
except Exception as e:
    print(f" ERREUR: {e}")
    access_token = None

# ÉTAPE 2: Tester les prédictions
if access_token:
    print(f"\n ÉTAPE 2: Tests de Prédiction")
    print("-" * 70)
    
    # Exemples de tests
    test_cases = [
        {"text": "This is a great article, thank you!", "expected": "Non-toxic"},
        {"text": "You are stupid and worthless!", "expected": "Toxic"},
        {"text": "I disagree with your opinion.", "expected": "Non-toxic"},
    ]
    
    headers = {"Authorization": f"Bearer {access_token}"}
    
    for i, test in enumerate(test_cases, 1):
        try:
            response = requests.post(
                f"{API_URL}/predict",
                headers=headers,
                json={"text": test["text"]},
                timeout=10
            )
            
            if response.status_code == 200:
                result = response.json()
                prediction = " Toxic" if result['is_toxic'] else " Non-toxic"
                print(f"\nTest {i}: {prediction} (Score: {result['score']}/100)")
                print(f"   Texte: \"{test['text']}\"")
                print(f"   Attendu: {test['expected']}")
            else:
                print(f"\n Erreur Test {i}: {response.status_code}")
                print(f"   {response.text}")
                
        except Exception as e:
            print(f"\n Erreur Test {i}: {e}")

### Test de l'API avec GKE

Testons l'API déployée sur Google Kubernetes Engine.

In [ ]:
import requests
import json

# CONFIGURATION PRODUCTION
PROD_API_URL = "http://34.22.130.34/docs"  # IP publique GKE
USERNAME = "admin"
PASSWORD = "admin"

# Test de santé
print(f"\n Health check de l'API")


try:
    health_response = requests.get(f"{PROD_API_URL}/health", timeout=10)
    
    if health_response.status_code == 200:
        health_data = health_response.json()
        print(f" API opérationnelle!")
        print(f"   Status: {health_data.get('status', 'N/A')}")
        print(f"   Model: {health_data.get('model', 'N/A')}")
    else:
        print(f"  Réponse inattendue: {health_response.status_code}")
        
except requests.exceptions.ConnectionError:
    print(f" ERREUR: Impossible de se connecter à {PROD_API_URL}")
    print(f"   L'API de production est peut-être arrêtée.")
except Exception as e:
    print(f" ERREUR: {e}")

# Authentification
print(f"\n Authentification JWT")
print("-" * 70)

try:
    # Note: L'endpoint /token n'est peut-être pas déployé en production
    token_response = requests.post(
        f"{PROD_API_URL}/token",
        data={"username": USERNAME, "password": PASSWORD},
        timeout=10
    )
    
    if token_response.status_code == 200:
        token_data = token_response.json()
        access_token = token_data['access_token']
        print(f" Token obtenu!")
        
        # Test de prédiction
        print(f"\n Test de Prédiction")
        print("-" * 70)
        
        headers = {"Authorization": f"Bearer {access_token}"}
        test_text = "This is a test comment."
        
        pred_response = requests.post(
            f"{PROD_API_URL}/predict",
            headers=headers,
            json={"text": test_text},
            timeout=10
        )
        
        if pred_response.status_code == 200:
            result = pred_response.json()
            print(f" Prédiction réussie!")
            print(f"   Texte: \"{test_text}\"")
            print(f"   Score: {result['score']}/100")
            print(f"   Toxic: {result['is_toxic']}")
        else:
            print(f" Erreur de prédiction: {pred_response.status_code}")
            
    elif token_response.status_code == 404:
        print(f"  Endpoint /token non trouvé (404)")
        print(f"   L'API de production doit être redéployée avec le endpoint /token")
    else:
        print(f" Erreur d'authentification: {token_response.status_code}")
        
except requests.exceptions.ConnectionError:
    print(f" Connexion impossible")
except Exception as e:
    print(f" ERREUR: {e}")

## Étape 4 : Sécurité et conformité RGPD

### Mesures de Sécurité Implémentées

Notre API respecte les standards de sécurité et la conformité RGPD :

**1. Authentification JWT (JSON Web Tokens)**
- Tokens signés avec secret cryptographique
- Expiration configurable (24h par défaut)
- Validation systématique des requêtes

**2. Validation des entrées**
- Pydantic models pour validation stricte
- Sanitization des inputs
- Protection contre injections

**3. RGPD**
- Anonymisation préalable des données (spaCy NER)
- Pas de stockage de données personnelles
- Registre de traitement documenté

**4. Sécurité Infrastructure**
- Service Account GCP avec permissions minimales
- Secrets dans Secret Manager (JWT_SECRET)
- Network policies Kubernetes

**5. Monitoring & Audit**
- Logs centralisés (Cloud Logging)
- Métriques de sécurité (tentatives auth échouées)
- Alertes sur comportements anormaux

In [ ]:
# Vérification des permissions IAM et Service Account
print(" Configuration IAM Kubernetes")
print("=" * 60)

iam_config = {
    "Service Account": "mlops-api-sa",
    "Namespace": "default",
    "Permissions GCP": [
        "storage.objects.get (Cloud Storage)",
        "logging.logEntries.create (Cloud Logging)",
        "monitoring.timeSeries.create (Cloud Monitoring)",
        "aiplatform.endpoints.predict (Vertex AI)"
    ],
    "Principe": "Least Privilege (permissions minimales)",
    "Workload Identity": "Recommandé pour production"
}

print(f"Service Account Kubernetes: {iam_config['Service Account']}")
print(f"\nPermissions GCP attribuées :")
for perm in iam_config['Permissions']:
    print(f" {perm}")

print(f"\n Principe appliqué: {iam_config['Principe']}")
print(f" {iam_config['Workload Identity']}")

print("\n Commandes pour vérifier IAM :")
print("gcloud iam service-accounts list")
print("kubectl get serviceaccount mlops-api-sa -o yaml")

## Étape 5 : Simulation de montée en charge avec Locust

### Objectifs des tests de charge

Notre stratégie de load testing vise à :
- **Mesurer les performances** : Latence P50, P95, P99
- **Identifier les limites** : Capacité maximale (RPS)
- **Détecter les goulots** : CPU, mémoire, I/O
- **Valider la scalabilité** : Horizontal Pod Autoscaling

### Configuration du test
- **Outil** : Locust (Python)
- **Scénarios** : Authentication + Prédictions (mix toxique/non-toxique)
- **Montée en charge** : Progressive (spawn rate contrôlée)
- **Métriques clés** : Response time, throughput, error rate

In [ ]:
# Vérification du fichier locustfile.py
import os

locust_file = "locustfile.py"
if os.path.exists(locust_file):
    print(" locustfile.py trouvé")
    print("\n Structure du test de charge :")
    
    # Lecture des principales métriques
    with open(locust_file, 'r', encoding='utf-8') as f:
        content = f.read()
        
    print("\n Scénarios de test détectés :")
    if "def login" in content:
        print("  Authentication JWT (/token)")
    if "def predict_toxicity" in content:
        print("  Prédictions toxicité (/predict)")
    if "def health_check" in content:
        print("  Health checks (/health)")
    print("  • Endpoint : http://localhost:8080 (ou production)")
else:
    print(" locustfile.py non trouvé")

### Exécution de la simulation de montée en charge

**Commande pour lancer Locust (mode headless) :**

```bash
# Test local (5 minutes, 50 utilisateurs max)
locust -f locustfile.py --host=http://localhost:8080 --users 50 --spawn-rate 5 --run-time 5m --headless --html report_loadtest.html

# Test production
locust -f locustfile.py --host=http://34.77.253.251 --users 100 --spawn-rate 10 --run-time 10m --headless --html report_production.html
```

**Commande pour interface Web (mode interactif) :**

```bash
locust -f locustfile.py --host=http://localhost:8080
# Ouvrir http://localhost:8089 dans le navigateur
```

**Métriques :**
- **P50 (Médiane)** : Temps de réponse pour 50% des requêtes
- **P95** : 95% des requêtes répondent sous ce temps (SLA typique)
- **P99** : 99% des requêtes (détection des outliers)
- **RPS (Requests/sec)** : Débit maximal supporté
- **Error Rate** : Taux d'erreur acceptable < 1%

## Étape 6 : Monitoring avec Google Cloud Monitoring

### Architecture de Monitoring

Notre solution de monitoring utilise **Google Cloud Monitoring** pour :

**1. Métriques Kubernetes/GKE (automatiques)**
- CPU/Memory des pods
- Network I/O
- Disk usage
- Pod restarts, crashes

**2. Métriques Applicatives (custom)**
- Request rate (requêtes/sec)
- Latency (P50, P95, P99)
- Error rate
- Token generation rate
- Model prediction count

**3. Logs centralisés**
- Cloud Logging (stdout/stderr des containers)
- Filtres et recherche avancée
- Alertes basées sur logs

**4. Alerting**
- Latency P95 > 500ms
- Error rate > 5%
- Pod availability < 2 replicas
- Memory usage > 80%